# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# read files from google drive
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/NTU_MLDS_Spring2018/HW2'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

# import modules from google drive
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

Mounted at /content/drive


# Download dataset

In [ ]:
# update gdown, if it failed to download large files
# !pip install --upgrade --no-cache-dir gdown

In [ ]:
# clr_conversation.txt
!gdown https://drive.google.com/uc?id=1JTwZ4RyEApT60v_CLHp7xAAfrirQQDEd

Downloading...
From: https://drive.google.com/uc?id=1JTwZ4RyEApT60v_CLHp7xAAfrirQQDEd
To: /content/clr_conversation.txt
100% 88.6M/88.6M [00:01<00:00, 84.6MB/s]


In [ ]:
# testing data
!gdown https://drive.google.com/uc?id=1I1oYmhtbYzVgDLVQJ1Uv9soGSiMyB5rD

Downloading...
From: https://drive.google.com/uc?id=1I1oYmhtbYzVgDLVQJ1Uv9soGSiMyB5rD
To: /content/test_input.txt
100% 314k/314k [00:00<00:00, 86.6MB/s]


In [ ]:
# baseline code / evaluation
# !gdown https://drive.google.com/uc?id=1pLREvo9ptzdg_NHgZESlj-vr1N60MpOF
# !tar -zxf evaluation.tar.gz  # don't show the progress

# Training

In [ ]:
%cd $GOOGLE_DRIVE_PATH/2-2

/content/drive/My Drive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2


In [ ]:
convo_file = '/content/clr_conversation.txt'
test_file = '/content/test_input.txt'

In [ ]:
# executing for about 6.2 hr (V100)
!python model_seq2seq.py --train_data $convo_file

Namespace(train_data='/content/clr_conversation.txt', num_epochs=200)
Building vocabulary...
Finished.
Vocabulary sizes: 10174
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10174, 1024, padding_idx=0)
    (gru): GRU(1024, 512, num_layers=2, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10174, 1024, padding_idx=0)
    (attn): Attention(
      (w_en): Linear(in_features=512, out_features=512, bias=True)
      (w_de): Linear(in_features=512, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (gru): GRU(1536, 512, num_layers=2, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=2048, out_features=10174, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
Epoch: 01 | Time: 1m 51s
	Train Loss: 5.434 | Train PPL: 229.173
	 Val. Loss: 5.542 |  Val. PPL: 255.210
Epoch: 02 | Time: 1m 50s
	Train Loss: 5.184 | Train PPL: 178.371
	 

Due to some bugs in the Colab ([OSError: [Errno 107] Transport endpoint is not connected](https://github.com/googlecolab/colabtools/issues/3441)), save the trained model under `/content` (to avoid triggering the error which would halt execution and session) and copy to the Google Drive after training.

In [ ]:
!cp -r /content/saved_model ./

# Inference

In [ ]:
# executing for about 13 m
!bash ./hw2_seq2seq.sh $test_file 'output.txt'

Namespace(test_data='/content/test_input.txt', output_file='output.txt')
output.txt saved.


# Evaluation

Baseline:  
* Perplexity < 100  
* Correlation Score > 0.45  


In [ ]:
%cd evaluation

/content/drive/MyDrive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2/evaluation


In [ ]:
# It seems problematic to execute main.py directly...
# !python main.py $test_file '../output.txt'

In [ ]:
from main import main
main(test_file, '../output.txt')

loading dictionary


/content/drive/MyDrive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2/evaluation/lm_module.py:104: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)  # modified for compatible with tensorflow 2
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/content/drive/MyDrive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2/evaluation/lm_module.py:115: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  log_prob = tf.layers.dense(encoder_outputs, vocab_size, name='output_projection')


Testing...
perplexity      : 37.635871 (baseline: < 100)


/content/drive/MyDrive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2/evaluation/cs_module.py:119: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x1 = Variable(x1,volatile=True).cuda() if use_cuda else Variable(x1,volatile=True)
/content/drive/MyDrive/Colab Notebooks/NTU_MLDS_Spring2018/HW2/2-2/evaluation/cs_module.py:120: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x2 = Variable(x2,volatile=True).cuda() if use_cuda else Variable(x2,volatile=True)


correlation score : 0.53139 (baseline: > 0.45)
